##### Info:
This tutorials aims to provide a short overview of an analysis of variance between land cover classes and their digital number values. The values were randomly created, as well as the labels per land cover class. 
This short tutorial is based on a blog post found here:

http://hamalg.blogspot.com/2015/11/python-for-data-analysis-part-16_23.html?m=1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

##### Generating random labels

In [2]:
np.random.seed(123)
classes = ["forest", "urban", "grassland", "water", "cropland"]
labels = np.random.choice(a = classes, p = [0.05, 0.15, 0.25, 0.05, 0.5], size = 10000)
labels

array(['cropland', 'grassland', 'grassland', ..., 'cropland', 'grassland',
       'cropland'], dtype='<U9')

##### Generating random digital numbers

In [3]:
dn_values = np.random.choice(500, size = 10000)
dn_values

array([266, 388, 495, ..., 101, 255, 229])

##### Creating dataframe

In [4]:
df = pd.DataFrame({'label': labels, 'dn': dn_values})
df.head(10)

,label,dn
0,cropland,266
1,grassland,388
2,grassland,495
3,cropland,164
4,cropland,390
5,grassland,237
6,cropland,404
7,cropland,128
8,water,103
9,grassland,285


##### Grouping data by label

In [5]:
groups = df.groupby('label').groups
groups

{'cropland': Int64Index([   0,    3,    4,    6,    7,   11,   15,   18,   19,   20,
             ...
             9982, 9983, 9986, 9987, 9988, 9989, 9994, 9996, 9997, 9999],
            dtype='int64', length=5035),
 'forest': Int64Index([  74,   87,  118,  147,  186,  194,  196,  209,  229,  235,
             ...
             9714, 9730, 9760, 9784, 9810, 9848, 9862, 9918, 9930, 9985],
            dtype='int64', length=504),
 'grassland': Int64Index([   1,    2,    5,    9,   10,   12,   14,   25,   26,   27,
             ...
             9968, 9969, 9974, 9975, 9978, 9980, 9990, 9992, 9995, 9998],
            dtype='int64', length=2422),
 'urban': Int64Index([  13,   16,   17,   30,   41,   50,   65,   68,   70,   77,
             ...
             9951, 9960, 9961, 9967, 9973, 9979, 9981, 9984, 9991, 9993],
            dtype='int64', length=1560),
 'water': Int64Index([   8,   33,   46,  138,  144,  184,  198,  215,  226,  241,
             ...
             9740, 9746, 9770, 9797, 9

##### Extracting the individual groups in an array

In [6]:
forest = dn_values[groups['forest']]
cropland = dn_values[groups['cropland']]
grassland = dn_values[groups['grassland']]
urban = dn_values[groups['urban']]
water = dn_values[groups['water']]

##### Performing the anova

In [7]:
stats.f_oneway(forest, cropland, grassland, urban, water)

F_onewayResult(statistic=0.30636815043537696, pvalue=0.8738792967978335)

##### Performing the post - hoc test

In [8]:
cid_pairs = []
for cid1 in range(4):
    for cid2 in range(cid1+1,5):
        cid_pairs.append((classes[cid1], classes[cid2]))

cid_pairs 

[('forest', 'urban'),
 ('forest', 'grassland'),
 ('forest', 'water'),
 ('forest', 'cropland'),
 ('urban', 'grassland'),
 ('urban', 'water'),
 ('urban', 'cropland'),
 ('grassland', 'water'),
 ('grassland', 'cropland'),
 ('water', 'cropland')]

In [9]:
for cid1, cid2 in cid_pairs:
    print(cid1, cid2)
    print(stats.ttest_ind(dn_values[groups[cid1]],
                         dn_values[groups[cid2]]))

forest urban
Ttest_indResult(statistic=0.6946848273953521, pvalue=0.48733110353888176)
forest grassland
Ttest_indResult(statistic=0.8501438110793602, pvalue=0.39531474163613944)
forest water
Ttest_indResult(statistic=0.45933375237769636, pvalue=0.6460962781592008)
forest cropland
Ttest_indResult(statistic=1.0325125965526665, pvalue=0.3018771450876062)
urban grassland
Ttest_indResult(statistic=0.1792471356757301, pvalue=0.8577527782462038)
urban water
Ttest_indResult(statistic=-0.12014927192464044, pvalue=0.904376750473604)
urban cropland
Ttest_indResult(statistic=0.4409467191153829, pvalue=0.65926602840313)
grassland water
Ttest_indResult(statistic=-0.24285213853493262, pvalue=0.8081371240447843)
grassland cropland
Ttest_indResult(statistic=0.28316963443813303, pvalue=0.7770546667547222)
water cropland
Ttest_indResult(statistic=0.3986207000779256, pvalue=0.6901881471791584)


In uneven datasets there is the need to adjust for the number of comparisons made by dividing the statistical significance level by the number of comparisons made. In our case we have a total number of 10 comparisons, and we are looking at a 5% level of significance, therefore we check p-values of the order 0.05/10 = 0.005 or less. This case of adjustment is called Bonferronni correction

##### Using Tukey post hoc

In [10]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

ModuleNotFoundError: No module named 'statsmodels'

##### Calculating the Tukey post hoc

In [ ]:
tukey = pairwise_tukeyhsd(endog = dn_values,
                         groups = labels,
                         alpha = 0.05) # Significant level


In [ ]:
tukey.plot_simultaneous()
plt.vlines(x = 0, ymin = 0, ymax = 5, color = 'green')

In [ ]:
tukey.summary()

##### Summary:
The output of a Tukey test, shows us the suggestion based on the data, if we should or not reject a null hypothesis (there are no differences between the mean values of the groups). This test above suggest that we cannot reject the null hypothesis in any of the comparisons.